1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.12，升级机器人电机驱动固件版本到0.2.10.11，详细使用请参考上位机使用说明
2. 请使用上位机调整机器人GR1T2电机pd参数（注意不是pid参数），
   参考值：上肢电机10-14/30-34电机 p：200，d：15;
          上肢电机15-16/35-36电机 p：100，d:10;
          腰部电机90              p:300，d:18;
          腰部电机91-92            p:500, d:20。
   可以根据软硬自己调节,腕关节可以小一点
3. 请使用指定的docker镜像进行开发，镜像名为fourier_hardware:v2.4，请使用开源仓库中的docker启动脚本启动docker

In [1]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# 核心类，用于对gr1t2 绝编进行标定以及计算offset (gr1t2的绝编设置特殊，不同于gr2t2 和 grmini）
# 请程序每次关闭之后对机器人进行断电（！！！！！非常重要）
t2_abs_helper = fourier_hardware_py.T2ABSHelper("/usr/local/config/fourier_hardware/gr1t2")
# 获取当前绝编位置，最多尝试3次，返回一个pair，代表成功与否，成功的话后面数组代表当前的绝编位置。
t2_abs_helper.getControlGroupABSAngle("waist",3)

unknown error happened . 


(True, array([2.94601011, 0.54341268, 4.02018022]))

**只有在标定时才需要调用（标定时为了保证精度，请使用插销固定关节）**

In [2]:
# 尝试对腰部绝编进行标定，标定将记录此刻的绝编位置，安装到上述路径下
# 对于t2，有以下几个control group需要标定：waist left_leg right_leg left_ankele right_ankle
t2_abs_helper.calibrateControlGroup("waist") #同样会返回是否标定成功

True

**使用t2_abs_helper 更新hardware_config_helper (这个操作本质会读取标定时绝编读数和机器人上电时绝编读数，修改hardware config中的offset，
后续位置更新由offset 和 相对编码器计算得出**

In [3]:
hardware_config_helper = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr1t2")
t2_abs_helper.updateControlGroupOffset("waist",hardware_config_helper)
# 请注意，此处必须返回True，才代表成功

True

In [4]:
t2_abs_helper.getControlGroupABSAngle("waist",3)

(True, array([2.94601011, 0.54456317, 4.02018022]))

In [6]:
# 尝试对所有绝编进行标定，注意，如果不关注一些control group，可以只标定部分，然后只更新部分control group的offset
# 只有在标定时再调用
t2_abs_helper.calibrateAllControlGroup()

hardware_config_helper = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr1t2")
abs_control_group = ["waist","left_leg","right_leg","left_ankle","right_ankle"]
for item in abs_control_group:
    update_flag = t2_abs_helper.updateControlGroupOffset(item,hardware_config_helper)
    print(item + " update offset:" + str(update_flag))
    print(item + " control group's joint:" + (str(t2_abs_helper.getControlGroupABSAngle(item,5)[1]) if (t2_abs_helper.getControlGroupABSAngle(item,5)[0]) else "get abs angle false"))
# 请注意，此处必须返回True，才代表成功

unknown error happened . 
waist update offset:True
waist control group's joint:[2.94601011 0.54494667 4.02018022]
left_leg update offset:True
left_leg control group's joint:[3.10976267 1.37291276 3.14849567 1.65900028]
unknown error happened . 
unknown error happened . 
right_leg update offset:True
right_leg control group's joint:[1.66283512 6.05653954 0.55568457 4.87614155]
left_ankle update offset:True
left_ankle control group's joint:[1.8327235  1.33226228]
right_ankle update offset:True
right_ankle control group's joint:[5.32751512 3.68308783]


In [7]:
hardware_config_helper.printHardwareConfig()

{"control_groups":[{"control_group_type":"t2_head","data_filter":true,"joints":["head_roll_joint","head_pitch_joint","head_yaw_joint"],"motors":[{"ctft":2.68,"filter_cf":3,"fri_scale":0.1,"ftc":1.4056048973281507,"fti":0.04874999967318416,"ftv":0.8515706976979021,"motor_name":"192.168.137.93","offset":0.0,"reverse_direction":false,"type":"revolute"},{"ctft":2.68,"filter_cf":3,"fri_scale":0.1,"ftc":2.0314999714106157,"fti":0.8453249951392948,"ftv":2.0710740949759314,"motor_name":"192.168.137.94","offset":0.0,"reverse_direction":false,"type":"revolute"},{"ctft":2.68,"filter_cf":3,"fri_scale":0.1,"ftc":2.006597377479171,"fti":0.5849999960996998,"ftv":0.6344,"motor_name":"192.168.137.95","offset":0.0,"reverse_direction":false,"type":"revolute"}],"motors_kind":"fsa","motors_mode":"pd","name":"head"},{"control_group_type":"direct","data_filter":true,"joints":["waist_yaw_joint","waist_pitch_joint","waist_roll_joint"],"motors":[{"ctft":2.68,"filter_cf":3,"fri_scale":0.1,"ftc":1.405604897328150

**对于gr1t2，每次hardware析构或者程序退出，请重新上电，否则位置将有误差**

In [8]:
# 配置各控制组的控制模式为PD模式，并忽略下肢控制的运动
# 腰部由于绝编和电机输出轴减速比过大，所以，在上电时，腰部位置尽量在中间，防止offset求取错误。
hardware_config_helper.changeControlGroupMotorMode("left_manipulator","pd")
hardware_config_helper.changeControlGroupMotorMode("right_manipulator","pd")
hardware_config_helper.changeControlGroupMotorMode("waist","pd")
hardware_config_helper.changeControlGroupMotorMode("left_wrist","pd")
hardware_config_helper.changeControlGroupMotorMode("right_wrist","pd")
hardware_config_helper.changeControlGroupMotorMode("left_leg","pd")
hardware_config_helper.changeControlGroupMotorMode("right_leg","pd")
hardware_config_helper.changeControlGroupMotorMode("left_ankle","pd")
hardware_config_helper.changeControlGroupMotorMode("right_ankle","pd")
hardware_config_helper.changeControlGroupMotorMode("head","ignore")
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
# ！！！！注意，t2 在 关闭程序之后需要重启机器。
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr1t2",hardware_config_helper)

In [21]:
hardware.disableRobot()

MOTOR: 192.168.137.74, DISABLE SUCCESS! 
MOTOR: 192.168.137.75, DISABLE SUCCESS! 
MOTOR: 192.168.137.70, DISABLE SUCCESS! 
MOTOR: 192.168.137.71, DISABLE SUCCESS! 
MOTOR: 192.168.137.72, DISABLE SUCCESS! 
MOTOR: 192.168.137.73, DISABLE SUCCESS! 
MOTOR: 192.168.137.10, DISABLE SUCCESS! 
MOTOR: 192.168.137.11, DISABLE SUCCESS! 
MOTOR: 192.168.137.12, DISABLE SUCCESS! 
MOTOR: 192.168.137.13, DISABLE SUCCESS! 
MOTOR: 192.168.137.14, DISABLE SUCCESS! 
MOTOR: 192.168.137.15, DISABLE SUCCESS! 
MOTOR: 192.168.137.16, DISABLE SUCCESS! 
MOTOR: 192.168.137.54, DISABLE SUCCESS! 
MOTOR: 192.168.137.55, DISABLE SUCCESS! 
MOTOR: 192.168.137.50, DISABLE SUCCESS! 
MOTOR: 192.168.137.51, DISABLE SUCCESS! 
MOTOR: 192.168.137.52, DISABLE SUCCESS! 
MOTOR: 192.168.137.53, DISABLE SUCCESS! 
MOTOR: 192.168.137.30, DISABLE SUCCESS! 
MOTOR: 192.168.137.31, DISABLE SUCCESS! 
MOTOR: 192.168.137.32, DISABLE SUCCESS! 
MOTOR: 192.168.137.33, DISABLE SUCCESS! 
MOTOR: 192.168.137.34, DISABLE SUCCESS! 
MOTOR: 192.168.1

True

[2025-03-27 13:43:24] [457] [error] left_ankle loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] left_leg loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] left_manipulator loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] left_wrist loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] right_ankle loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] right_leg loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] right_manipulator loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] right_wrist loss message more than 50 times .
[2025-03-27 13:43:24] [457] [error] waist loss message more than 50 times .
[2025-03-27 13:43:25] [457] [error] left_ankle loss message more than 50 times .
[2025-03-27 13:43:25] [457] [error] left_leg loss message more than 50 times .
[2025-03-27 13:43:25] [457] [error] left_manipulator loss message more than 50 times .
[2025-03-27 13:43

**请在标定结束后断电重新上电**